# V889 Her Flare Detection Analysis

Flare detection and analysis for V889 Her using TESS data


In [ ]:
import os
import re
import sys
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set the project root
PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name in ['notebooks', 'src']:
    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))
from src.flarepy_V889_Her import FlareDetector_V889_Her


## Load data (all sectors)


In [ ]:
# Data folder path
folder_path = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

# Load all FITS files
file_list = sorted(folder_path.glob("*.fits"))
print(f"Found {len(file_list)} FITS files")

# Create a detector for each sector
detectors = {}
for file_path in file_list:
    match = re.search(r'-s(\d{4})-', file_path.name)
    if match:
        sector = f"s{match.group(1)}"
        key = f"detector_{sector}"
        detectors[key] = FlareDetector_V889_Her(file=str(file_path), process_data=True)
        print(f"Loaded {sector}: {detectors[key].flare_number} flares detected")

print(f"\nTotal detectors: {len(detectors)}")


## Results summary


In [ ]:
sum_flare_number = sum(d.flare_number for d in detectors.values())
sum_obs_time = sum(d.precise_obs_time for d in detectors.values())

print(f"Total flares: {sum_flare_number}")
print(f"Total observation time: {sum_obs_time:.2f} days")
if sum_obs_time > 0:
    print(f"Average flare frequency: {sum_flare_number / sum_obs_time:.2f} flares/day")


## Visualization

### Light curve


In [ ]:
# Plot light curves for each sector
for name, detector in list(detectors.items())[:3]:
    detector.plt_flare(title=f"V889 Her {name}")


### Rotation period estimation (Gaussian fitting)


In [ ]:
# Estimate the period for a representative sector (with diagnostic plots)
sample_detector = list(detectors.values())[0]
sample_detector.rotation_period(show_plot=True)

print(f"\nEstimated period: {sample_detector.per:.4f} (+{sample_detector.per_err_plus:.4f} / -{sample_detector.per_err_minus:.4f}) days")


### Energy distribution


In [ ]:
# Energy distribution for each sector
for detector in list(detectors.values())[:3]:
    detector.plot_energy()


## Statistical analysis (class variables)


In [ ]:
# Correlation between time and amplitude
fig = px.scatter(
    x=FlareDetector_V889_Her.array_observation_time,
    y=FlareDetector_V889_Her.array_amplitude,
    title="Observation time vs brightness variation amplitude",
    labels={"x": "Observation Time (BJD)", "y": "Amplitude"}
)
fig.show()


In [ ]:
# Correlation between period and flare frequency
fig = px.scatter(
    x=FlareDetector_V889_Her.array_per,
    y=FlareDetector_V889_Her.array_flare_ratio,
    title="Rotation period vs flare frequency",
    labels={"x": "Period (days)", "y": "Flare Rate (flares/day)"}
)
fig.show()


## Paper-quality figures


In [ ]:
import matplotlib.pyplot as plt

# Paper-quality settings
plt.rcParams['xtick.major.width'] = 1.5
plt.rcParams['ytick.major.width'] = 1.5
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['pdf.fonttype'] = 42

# High-quality plot for a representative sector
sample = list(detectors.values())[0]
sample.plt_flare(
    title="V889 Her Light Curve",
    save_path=str(PROJECT_ROOT / "outputs" / "V889_Her_lightcurve.pdf")
)
